# Data Preprocessing

The preprocessing phase will take x steps:

1. Setup - import packages, setup paths
2. Import Data - bring in the data
3. Cleaning Data - aggregate data as necessary and make transformations as required

## Setup

---

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


#### Imports

In [458]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

#### Paths

In [3]:
path_proj = 'U:/projects/donor_pred/preprocessing'
path_data = 'U:/data'

#### Globals

In [4]:
FY_START = 11
FY_END = 19

## Import Data
---

#### Files Needed

In [5]:
d_files = 'donors_fy08-present.csv'
t_files_fys = range(11, 20)
t_files_series = ['Chamber', 'Clx', 'Connections', 'Family', 
                  'Organ', 'Pops', 'Specials', 'Summer']
a_file_names = ['age', 'est_hh_income', 'len_of_residence', 'propensity_rating']

In [39]:
t_files = [series + str(fy) + '.csv' for series in t_files_series for fy in t_files_fys]
a_files = [name for name in a_file_names]

### Donor Data

In [7]:
def donor_data_import(path_to_data_files, file):
    donor_raw = pd.read_csv(path_to_data_files + '/donors/' + file, encoding='ISO-8859-1')
    return donor_raw

In [8]:
d_data = donor_data_import(path_data, d_files)

C:\Users\bkessler\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [9]:
donor_fy = d_data.campaign.str[3:8] # parse last 5 characters of campaign column
d_data['fy'] = donor_fy # append that to a column named 'fy'

Identify which 'fys' don't start with an integer, these we can remove. Also, identify the years that are outside of the range in question

In [10]:
filter_fys = {fy: False for fy in d_data.fy.drop_duplicates()}
for fy,boo in filter_fys.items():
    try:
        int(fy[0])
        if (int(fy[3:]) > FY_START) & (int(fy[3:]) < FY_END+1):
            filter_fys[fy] = True
        else:
            continue
    except:
        continue

Map the fy column to identify rows to remove then remove them

In [11]:
d_data['keep'] = d_data['fy'].map(filter_fys)
donor_df = d_data.loc[d_data.keep].reset_index()
donor_df.drop(columns=['keep'], inplace=True)

Convert dates to datetime

In [12]:
donor_df['cont_dt'] = pd.to_datetime(donor_df['cont_dt'])

Remove unnecessary columns

In [13]:
donor_cols = ['summary_cust_id', 'customer_no', 'gift_plus_pledge', 'cont_dt', 'fy', 'ps_sol', 'sum_ps_sol']
donor_df = donor_df[donor_cols]

In [14]:
donor_df.sample(10)

,summary_cust_id,customer_no,gift_plus_pledge,cont_dt,fy,ps_sol,sum_ps_sol
13891,2454548,2454548,2000.0,2014-05-02,13-14,Michael Lawler,Michael Lawler
46204,153459,2857736,0.0,2016-02-16,18-19,Gary Good,Gary Good
29610,3212346,3212346,50.0,2016-03-14,15-16,NaN,NaN
30050,2750149,2750149,100.0,2015-12-10,15-16,NaN,NaN
19733,380818,380818,250.0,2014-10-01,14-15,Hilary Zappin,Hilary Zappin
47432,114765,114765,50.0,2019-03-14,18-19,Default Solicitor,Default Solicitor
48031,110806,110806,15.0,2019-02-01,18-19,Default Solicitor,Default Solicitor
15991,2853230,2853230,-312.5,2013-10-21,13-14,Michael Lawler,Michael Lawler
44823,2411501,2411501,0.0,2018-06-11,17-18,Nikki Palley,Nikki Palley
16065,3082944,3082944,25.0,2013-12-28,13-14,NaN,NaN


Strip FY to only include the last two digits of fiscal year

In [15]:
donor_df['fy'] = donor_df['fy'].str[3:]

#### Identify first Pledge

(COMMENTED OUT FOR NOW)

In [16]:
# first_donation = donor_df.groupby(['summary_cust_id', 'customer_no', 'cont_dt', 'fy']).sum().reset_index()

In [17]:
# first_donation = first_donation.sort_values(by=['summary_cust_id', 'cont_dt']).drop_duplicates(subset=['summary_cust_id'], keep='first').reset_index(drop=True)

In [18]:
# first_donation.head()

#### Agg Donor info

In [294]:
d_manip = donor_df.sort_values(['cont_dt'], ascending=False).groupby(['summary_cust_id']).agg({
    'gift_plus_pledge': ['sum', 'mean'],
    'ps_sol': [(lambda x: x.tail(1))],
    'fy': ['count']
}).reset_index()

In [295]:
d_manip.columns = ['summary_cust_id', 'gift_plus_pledge_total', 'gift_plus_pledge_avg', 'ps_sol', 'years_of_giving']

In [296]:
d_fin = d_manip.copy()

In [297]:
d_fin['per_year_giving'] = d_fin['gift_plus_pledge_total'] / d_fin['years_of_giving']

In [298]:
d_fin['solicitor'] = ~d_fin['ps_sol'].isna() * 1

In [299]:
d_fin['donor'] = 1

## Ticketing Data

In [20]:
dtypes = {
    'section': str,
    'summary_cust_name': str
}

In [21]:
def ticketing_data_import(path_to_data_files, filename, dtype=dtypes):
    df = pd.read_csv(path_data + '/ticketing/' + filename, skiprows=3, dtype=dtype)
    return df

In [22]:
t_data = pd.concat([ticketing_data_import(path_data, file) for file in t_files])

C:\Users\bkessler\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3296: DtypeWarning: Columns (6,23) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Keep Appropriate Columns in a new DF called t_manip

In [23]:
t_cols = ['summary_cust_id', 'customer_no', 'perf_dt', 
          'paid_amt', 'price_type_group', 'season_desc', 'attended']

In [24]:
t_manip = t_data[t_cols].dropna(subset=['summary_cust_id'])

#### Remove Bad Ids

In [25]:
bad_ids = [
    0,                 # Unknown IDs
    2700674,           # Terry Dwyer
    955085,            # PSO Comps
    3141490,           # Symphony Shop
    91013,             # PSO Orchestra Members
    118401,            # PSO Prez - JF
    91006,             # PSO Artist Comps
    3328612,           # Development Guest
    925728,            # Kurt Mortenson (Internal)
    2010347,           # Goldstar\n",
    2437127,           # Lorraine Caukin (Internal)
    2515897,           # Gregory Pierre Cox (internal)
    3080718,           # Gary Good
    91015,             # PSO Press
    120696             # Carl St. Clair
]

In [26]:
t_manip = t_manip.loc[~t_manip['summary_cust_id'].isin(bad_ids)].reset_index(drop=True)

#### Add a series column

In [27]:
series = t_manip['season_desc'].str.split().str[-1]

In [28]:
t_manip['series'] = series

#### Add columns for Subscription, Flex, Single, Comp

In [29]:
t_manip.loc[t_manip['price_type_group'] == 'Subscription', 'subs'] = 1
t_manip.loc[(t_manip['price_type_group'] == 'Single ') | 
            (t_manip['price_type_group'] == 'Discount') | 
            (t_manip['price_type_group'] == 'Discount'), 'single'] = 1
t_manip.loc[t_manip['price_type_group'] == 'Flex', 'flex'] = 1
t_manip.loc[t_manip['price_type_group'] == 'Comp', 'comp'] = 1

#### Add columns for Series Types (classics, pops, summer, family, specials, other)

In [30]:
t_manip.series.value_counts()

Classics       408436
Pops           240055
Summer         227428
Family         124303
Specials        91917
Connections     37272
Organ           26577
Chamber          6962
Name: series, dtype: int64

In [219]:
t_manip.loc[t_manip['price_type_group'] == 'Classics', 'classics'] = 1
t_manip.loc[t_manip['price_type_group'] == 'Pops', 'pops'] = 1
t_manip.loc[t_manip['price_type_group'] == 'Summer', 'summer'] = 1
t_manip.loc[t_manip['price_type_group'] == 'Family', 'family'] = 1
t_manip.loc[t_manip['price_type_group'] == 'Specials', 'specials'] = 1
t_manip.loc[~t_manip['price_type_group'].isin(
    ['Classics', 'Pops', 'Summer', 'Family', 'Specials']
), 'other'] = 1

t_manip.loc[t_manip['attended'] == 'Attended', 'attend'] = 1
t_manip['attend'].fillna(0, inplace=True)

In [220]:
t_manip.head()

,summary_cust_id,customer_no,perf_dt,paid_amt,price_type_group,season_desc,attended,series,subs,single,flex,comp,classics,pops,summer,family,specials,other,attend
0,12170.0,12170.0,10/31/2010 3:00:00 PM,51.0,Subscription,PS 10-11 Chamber,Attended,Chamber,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
1,12170.0,12170.0,10/31/2010 3:00:00 PM,51.0,Subscription,PS 10-11 Chamber,Attended,Chamber,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
2,22883.0,22883.0,10/31/2010 3:00:00 PM,66.0,Subscription,PS 10-11 Chamber,NaN,Chamber,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
3,22883.0,22883.0,10/31/2010 3:00:00 PM,66.0,Subscription,PS 10-11 Chamber,NaN,Chamber,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0
4,22883.0,22883.0,10/31/2010 3:00:00 PM,66.0,Subscription,PS 10-11 Chamber,NaN,Chamber,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0


In [245]:
t_fin = t_manip.groupby(['summary_cust_id', 'customer_no']).agg({
    'paid_amt': ['sum'],
    'attend': ['sum', 'count'],
    'subs': ['sum'],
    'single': ['sum'],
    'flex': ['sum'],
    'comp': ['sum'],
    'classics': ['sum'],
    'pops': ['sum'],
    'summer': ['sum'],
    'family': ['sum'],
    'specials': ['sum'],
    'other': ['sum']
}).reset_index()

In [246]:
t_fin[('paid_amt', 'true_mean')] = t_fin[('paid_amt', 'sum')] / (t_fin[('attend', 'count')] - t_fin[('comp', 'sum')])

In [247]:
t_fin.columns = ['summary_cust_id', 'customer_no', 'tot_paid', 'attended', 'tickets', 'subs',
                'single', 'flex', 'comp', 'classics', 'pops', 'summer', 'family', 'specials',
                'other', 'avg_paid']

In [248]:
t_fin = t_fin[['summary_cust_id', 'customer_no', 'attended', 'tickets', 'subs',
                'single', 'flex', 'comp', 'classics', 'pops', 'summer', 'family', 'specials',
                'other', 'avg_paid']]

## Attributes

In [47]:
def attribute_data_import(path_to_data_files, filenames, pull_date):
    main = pd.read_csv(path_data + '/attributes/' + filenames[0] + '_' + pull_date + '.csv',
                      skiprows=7)
    main = main[['customer_no']].drop_duplicates()
    
    
    for filename in filenames:
        df = pd.read_csv(path_data + '/attributes/' + filename + '_' + pull_date + '.csv', 
                         skiprows=7)
        df = df[['customer_no', 'key_value']]
        df.rename(columns={"customer_no": "customer_no", "key_value": filename}, inplace=True)
    
        main = main.merge(df, on='customer_no', how='left')
    return main

In [177]:
a_data = attribute_data_import(path_data, a_files, '8-21-19')

In [193]:
a_manip = a_data.dropna(subset=['est_hh_income']).reset_index(drop=True)

#### Use Imputer to fill missing vals

Starting with age - 
* create imputer with missing values = 0
* for the imputer to the pandas dataframe with age as the only column
* create a new column and set the age imputed to that column

In [194]:
age_imp = SimpleImputer(missing_values=0, strategy='mean')

In [195]:
age_imputed = age_imp.fit_transform(a_manip[['age']])

In [196]:
a_manip['age_imp'] = age_imputed

Then do length of residence

In [197]:
a_manip['len_of_residence'].fillna(' ', inplace=True)

In [200]:
residence_imp = SimpleImputer(missing_values=' ', strategy='most_frequent')

In [201]:
residence_imputed = residence_imp.fit_transform(a_manip[['len_of_residence']])

In [202]:
a_manip['len_residence_imp'] = residence_imputed

Check if there are any NAs remaining

In [338]:
a_manip = a_manip[['customer_no', 'est_hh_income', 'age_imp', 'len_residence_imp']]

In [339]:
a_manip.isna().any()

customer_no          False
est_hh_income        False
age_imp              False
len_residence_imp    False
dtype: bool

## Merge DFs

DFs = a_manip, t_fin, d_fin

In [446]:
working_data = t_fin.merge(a_manip, on='customer_no', how='left')

In [447]:
working_data = working_data.merge(d_fin[['summary_cust_id', 
                                         'per_year_giving', 
                                         'solicitor', 
                                         'donor']], on='summary_cust_id', how='left')

In [448]:
working_data.dropna(subset=['age_imp'], inplace=True)

Fill NAs from avg_paid, per_year_giving, solicitor, donor to 0

In [449]:
working_data = working_data.fillna(0).reset_index(drop=True)

In [450]:
working_data = working_data.loc[working_data['per_year_giving'] > 0]

In [468]:
processed_df = pd.concat([working_data.drop(['summary_cust_id', 'customer_no', 'donor', 'est_hh_income'], axis=1), 
                          pd.get_dummies(working_data['est_hh_income'], prefix='hh_income')], 
                         axis=1).reset_index(drop=True)

In [469]:
processed_df.shape

(2726, 26)

## Build Model

In [470]:
processed_df['per_year_giving'] = processed_df['per_year_giving']

In [471]:
annual_giving = processed_df['per_year_giving']
features = processed_df.drop('per_year_giving', 
                             axis=1) # axis indicates a column (0 for row, 1 for column)

X_train, X_test, y_train, y_test = train_test_split(features, annual_giving, 
                                                    test_size=0.2)

Standard Scaler for features

In [472]:
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

C:\Users\bkessler\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\bkessler\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\bkessler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [473]:
regr = LinearRegression()
regr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [474]:
est = sm.OLS(y_train, X_train.astype(float)).fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:        per_year_giving   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                     26.73
Date:                Thu, 22 Aug 2019   Prob (F-statistic):           2.73e-81
Time:                        13:38:38   Log-Likelihood:                -17861.
No. Observations:                2180   AIC:                         3.576e+04
Df Residuals:                    2162   BIC:                         3.586e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1           -24.7717     61.894     -0.400      0.6